In [1]:
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install bitsandbytes
!pip install peft
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.9 MB/s eta 0:00:00


In [1]:
from huggingface_hub import notebook_login

notebook_login()

### Load Dataset

In [3]:
from datasets import load_dataset, Dataset

dataset = load_dataset("garage-bAInd/Open-Platypus")
dataset

Generating train split:   0%|          | 0/24926 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction', 'data_source'],
        num_rows: 24926
    })
})

In [4]:
val_set_size = 0.1

dataset = dataset["train"].train_test_split(test_size=val_set_size,
                                            shuffle=True, seed=2)
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction', 'data_source'],
        num_rows: 22433
    })
    test: Dataset({
        features: ['input', 'output', 'instruction', 'data_source'],
        num_rows: 2493
    })
})

In [5]:
train_dataset = dataset["train"]
valid_dataset = dataset["test"]

In [6]:
train_df = train_dataset.to_pandas()
valid_df = valid_dataset.to_pandas()

train_df

,input,output,instruction,data_source
0,,We plot $r = 1 + \cos \theta.$ If we rotate i...,The polar curve $r = 1 + \cos \theta$ is rotat...,MATH/PRM-800K
1,,"Because the process is adiabatic, $q=0$, and $...","In this example, $2.50 \mathrm{~mol}$ of an id...",scibench
2,,$38 \mathrm{~N} / \mathrm{C}$,A charge of $20 \mathrm{nC}$ is uniformly dis...,scibench
3,,The next border requires an additional $6\time...,Thirteen blue and six green hexagonal tiles we...,MATH/PRM-800K
4,,\ndef max_increase_keeping_skyline(grid):\n ...,There are `n` persons on a social media websit...,leetcode_ne
...,...,...,...,...
22428,"Choose A, B, C or D as your solution.",B,"Advertisement: HomeGlo Paints, Inc. , has won ...",reclor
22429,,"To solve this problem, I need to convert the u...","On a hot day, Megan likes to eat a Popsicle ev...",MATH/PRM-800K
22430,"Choose A, B, C or D as your solution.",A,The state of Eastern Seaboard enacted a sales ...,ARB
22431,,"This problem involves rational expressions, so...",Find the sum of all $x$ that satisfy the equat...,MATH/PRM-800K


In [7]:
def generate_text(df):
    df["text"] = df[["instruction", "output"]].apply(lambda x: x['instruction'] + " " + x["output"], axis=1)
    df.drop(columns=["instruction", "output"], inplace=True)
    return Dataset.from_pandas(df)

In [8]:
train_data = generate_text(train_df)
valid_data = generate_text(valid_df)

In [9]:
model_ckpt_path = "meta-llama/Llama-2-7b-hf"

### Tokenizer

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckpt_path)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

### Bits And Bytes Config

In [11]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype="float16",
                                bnb_4bit_use_double_quant=True)

### Model

In [12]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments

model = AutoModelForCausalLM.from_pretrained(model_ckpt_path,
                                             quantization_config=bnb_config,
                                             device_map={"": 0})

model.config.use_cache=False
model.config.pretraining_tp=1

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

### Lora Config

In [27]:
from peft import LoraConfig

peft_config = LoraConfig(r=16,
                         lora_alpha=16,
                         target_modules=["gate_proj", "down_proj", "up_proj"],
                         lora_dropout=0.05,
                         bias="none",
                         task_type="CAUSAL_LM")

### Training Arguments

In [28]:
from transformers import TrainingArguments

output_dir = "platypus_llama_7b"
training_arguments = TrainingArguments(output_dir=output_dir,
                                       per_device_train_batch_size=8,
                                       per_device_eval_batch_size=8,
                                       gradient_accumulation_steps=4,
                                       optim="paged_adamw_32bit",
                                       learning_rate=2e-4,
                                       lr_scheduler_type="cosine",
                                       save_strategy="steps",
                                       save_steps=50,
                                       save_total_limit=100,
                                       logging_steps=10,
                                       num_train_epochs=1,
                                       max_steps=110,
                                       fp16=True,
                                       push_to_hub=True)

### Trainer

In [29]:
from trl import SFTTrainer

trainer = SFTTrainer(model=model,
                     train_dataset=train_data,
                     eval_dataset=valid_data,
                     peft_config=peft_config,
                     dataset_text_field="text",
                     args=training_arguments,
                     tokenizer=tokenizer,
                     packing=False,
                     max_seq_length=512)

Map:   0%|          | 0/22433 [00:00<?, ? examples/s]

Map:   0%|          | 0/2493 [00:00<?, ? examples/s]

In [30]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: ignored

In [ ]:
trainer.push_to_hub()

In [ ]:
trainer.save_model("platypus_llama_7b")

In [31]:
import os

output_dir = os.path.join("platypus_llama_7b", "final_checkpoint")
trainer.model.save_pretrained(output_dir)